# Check implementation of Hamberg et al model

In [1]:
import os

import arviz as az
import chi
import chi.plots
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.figure_factory as ff
import plotly.graph_objects as go
import xarray as xr

from model import define_hamberg_model, define_hamberg_population_model

## Define Hamberg model

In [2]:
mechanistic_model, parameters_df = define_hamberg_model()

## Define covariate model (ignoring other sources of IIV)

In [3]:
def get_typical_parameters(age, cyp2c9, vkorc1):
    """
    Returns typical model parameters for patient with input characteristics
    according to Hamberg et al.

    :param age: The age of the patient.
    :type age: int
    :param cyp2c9: One of 5 variants of the CYP2C9 gene:
        ['*1/*1', '*1/*2', '*1/*3', '*2/*2', '*2/*3','*3/*3']
    :type cyp2c9: str
    :param vkorc1: One of 5 variants of the VKORC1 gene:
        ['GG', 'GA', 'AA']
    :type vkorc1: str
    """
    cyp2c9_variants = ['*1/*1', '*1/*2', '*1/*3', '*2/*2', '*2/*3','*3/*3']
    if cyp2c9 not in cyp2c9_variants:
        raise ValueError(
            'The provided CYP2C9 genotype is not among the allowed variants.')
    vkorc1_variants = ['GG', 'GA', 'AA']
    if vkorc1 not in vkorc1_variants:
        raise ValueError(
            'The provided VKORC1 genotype is not among the allowed variants.')

    # Define covariate independent parameters
    effective_volume_central = 14.3           # in L
    transition_rate_chain_1 = 3 / 28.6        # in 1/h
    transition_rate_chain_2 = 3 / 118.3       # in 1/h

    # Compute clearance
    clearance_per_allele_1 = 0.174            # in L/h
    clearance_per_allele_2 = 0.0879           # in L/h
    clearance_per_allele_3 = 0.0422           # in L/h

    if cyp2c9 == '*1/*1':
        clearance = 2 * clearance_per_allele_1
    if cyp2c9 == '*1/*2':
        clearance = clearance_per_allele_1 + clearance_per_allele_2
    if cyp2c9 == '*1/*3':
        clearance = clearance_per_allele_1 + clearance_per_allele_3
    if cyp2c9 == '*2/*2':
        clearance = 2 * clearance_per_allele_2
    if cyp2c9 == '*2/*3':
        clearance = clearance_per_allele_2 + clearance_per_allele_3
    if cyp2c9 == '*3/*3':
        clearance = 2 * clearance_per_allele_3

    change_per_year = -0.00571              # rel. change centered around 71 y
    clearance *= (1 + change_per_year * (age - 71))

    # Compute EC50 of warfarin
    ec50_per_allele_A = 0.96                # in mg/L
    ec50_per_allele_G = 2.05                # in mg/L

    if vkorc1 == 'GG':
        half_maximal_effect_conc = 2 * ec50_per_allele_G
    if vkorc1 == 'GA':
        half_maximal_effect_conc = ec50_per_allele_G + ec50_per_allele_A
    if vkorc1 == 'AA':
        half_maximal_effect_conc = 2 * ec50_per_allele_A

    parameters = [
        clearance / effective_volume_central,
        half_maximal_effect_conc,
        transition_rate_chain_1,
        transition_rate_chain_2,
        effective_volume_central
    ]

    return parameters


## Reproduce Figure 3 in Hamberg et al

In [4]:
# Define simulation conditions
times = np.linspace(0, 30, num=1000) * 24

# I: Simulate treatment response for 70 year old infividuals and different
# CYP2C9 variants and the GG VKORC1 genotype for 2.5 target INR
age = 70
cyp2c9_variants = ['*1/*1', '*2/*2', '*3/*3']
vkorc1_variant = 'GG'
doses = [7.6, 3.8, 1.8]
parameters = []
for cyp2c9_variant in cyp2c9_variants:
    parameters.append(get_typical_parameters(
        age, cyp2c9_variant, vkorc1_variant))

simulation1 = []
for idp, params in enumerate(parameters):
    # Set dosing regimen
    # (Model only captures S-warfarin, so need to divide dose by 2)
    dose = doses[idp]
    mechanistic_model.set_dosing_regimen(dose, start=0, period=24)

    # Simulate response for 2.5 target INR
    simulation1.append(mechanistic_model.simulate(params, times))

# II: Simulate treatment response for 70 year old infividuals and different
# CYP2C9 variants and the GA VKORC1 genotype for 2.5 target INR
age = 70
cyp2c9_variants = ['*1/*1', '*2/*2', '*3/*3']
vkorc1_variant = 'GA'
doses = [5.6, 2.8, 1.4]
parameters = []
for cyp2c9_variant in cyp2c9_variants:
    parameters.append(get_typical_parameters(
        age, cyp2c9_variant, vkorc1_variant))

simulation2 = []
for idp, params in enumerate(parameters):
    # Set dosing regimen
    dose = doses[idp]
    mechanistic_model.set_dosing_regimen(dose, start=0, period=24)

    # Simulate response for 2.5 target INR
    simulation2.append(mechanistic_model.simulate(params, times))

# III: Simulate treatment response for 70 year old infividuals and different
# CYP2C9 variants and the GA VKORC1 genotype for 2.5 target INR
age = 70
cyp2c9_variants = ['*1/*1', '*2/*2', '*3/*3']
vkorc1_variant = 'AA'
doses = [3.6, 1.8, 0.9]
parameters = []
for cyp2c9_variant in cyp2c9_variants:
    parameters.append(get_typical_parameters(
        age, cyp2c9_variant, vkorc1_variant))

simulation3 = []
for idp, params in enumerate(parameters):
    # Set dosing regimen
    dose = doses[idp]
    mechanistic_model.set_dosing_regimen(dose, start=0, period=24)

    # Simulate response for 2.5 target INR
    simulation3.append(mechanistic_model.simulate(params, times))

# Figure 1: Treatment response of patients with different CYP2C9 genptypes for
# 2.5 INR maintenance dose
times /= 24
fig = go.Figure()
fig.add_trace(go.Scatter(
    x = times,
    y = simulation1[0][1],
    name = '*1/*1 + GG'
))
fig.add_trace(go.Scatter(
    x = times,
    y = simulation1[1][1],
    name = '*2/*2 + GG'
))
fig.add_trace(go.Scatter(
    x = times,
    y = simulation1[2][1],
    name = '*3/*3 + GG'
))
fig.update_layout(
    xaxis_title='Time in days',
    yaxis_title='Prothrombin time in INR',
)
fig.show()

# Figure 2: Treatment response of patients with different CYP2C9 genptypes for
# 2.5 INR maintenance dose
fig = go.Figure()
fig.add_trace(go.Scatter(
    x = times,
    y = simulation1[0][0],
    name = '*1/*1 + GG'
))
fig.add_trace(go.Scatter(
    x = times,
    y = simulation1[1][0],
    name = '*2/*2 + GG'
))
fig.add_trace(go.Scatter(
    x = times,
    y = simulation1[2][0],
    name = '*3/*3 + GG'
))
fig.update_layout(
    xaxis_title='Time in days',
    yaxis_title='Warfarin concentration in mg/L',
)
fig.show()

# Figure 3: Treatment response of patients with different CYP2C9 genptypes for
# 2.5 INR maintenance dose
fig = go.Figure()
fig.add_trace(go.Scatter(
    x = times,
    y = simulation2[0][1],
    name = '*1/*1 + GA'
))
fig.add_trace(go.Scatter(
    x = times,
    y = simulation2[1][1],
    name = '*2/*2 + GA'
))
fig.add_trace(go.Scatter(
    x = times,
    y = simulation2[2][1],
    name = '*3/*3 + GA'
))
fig.update_layout(
    xaxis_title='Time in days',
    yaxis_title='Prothrombin time in INR',
)
fig.show()

# Figure 4: Treatment response of patients with different CYP2C9 genptypes for
# 2.5 INR maintenance dose
fig = go.Figure()
fig.add_trace(go.Scatter(
    x = times,
    y = simulation2[0][0],
    name = '*1/*1 + GA'
))
fig.add_trace(go.Scatter(
    x = times,
    y = simulation2[1][0],
    name = '*2/*2 + GA'
))
fig.add_trace(go.Scatter(
    x = times,
    y = simulation2[2][0],
    name = '*3/*3 + GA'
))
fig.update_layout(
    xaxis_title='Time in days',
    yaxis_title='Prothrombin time in INR',
)
fig.show()

# Figure 5: Treatment response of patients with different CYP2C9 genptypes for
# 2.5 INR maintenance dose
fig = go.Figure()
fig.add_trace(go.Scatter(
    x = times,
    y = simulation3[0][1],
    name = '*1/*1 + AA'
))
fig.add_trace(go.Scatter(
    x = times,
    y = simulation3[1][1],
    name = '*2/*2 + AA'
))
fig.add_trace(go.Scatter(
    x = times,
    y = simulation3[2][1],
    name = '*3/*3 + AA'
))
fig.update_layout(
    xaxis_title='Time in days',
    yaxis_title='Prothrombin time in INR',
)
fig.show()

# Figure 6: Treatment response of patients with different CYP2C9 genptypes for
# 2.5 INR maintenance dose
fig = go.Figure()
fig.add_trace(go.Scatter(
    x = times,
    y = simulation3[0][0],
    name = '*1/*1 + AA'
))
fig.add_trace(go.Scatter(
    x = times,
    y = simulation3[1][0],
    name = '*2/*2 + AA'
))
fig.add_trace(go.Scatter(
    x = times,
    y = simulation3[2][0],
    name = '*3/*3 + AA'
))
fig.update_layout(
    xaxis_title='Time in days',
    yaxis_title='Prothrombin time in INR',
)
fig.show()


## Plot steady state distribution of INR for a 5mg warfarin treatment

### Generate demographic data

In [5]:
# Define population model
population_model = define_hamberg_population_model()

# Define covariates
# Frequency of alleles matches dataset in publication
n_ids = 1000
n_cov = 3
covariates = np.zeros(shape=(n_ids, n_cov))

n_cyp2p9_33 = int(np.ceil(0.006 * n_ids))
covariates[:n_cyp2p9_33, 0] += 1
n_cyp2p9_23 = int(np.ceil(0.012 * n_ids))
covariates[:n_cyp2p9_33+n_cyp2p9_23, 0] += 1
n_cyp2p9_22 = int(np.ceil(0.014 * n_ids))
covariates[:n_cyp2p9_33+n_cyp2p9_23+n_cyp2p9_22, 0] += 1
n_cyp2p9_13 = int(np.ceil(0.123 * n_ids))
covariates[:n_cyp2p9_33+n_cyp2p9_23+n_cyp2p9_22+n_cyp2p9_13, 0] += 1
n_cyp2p9_12 = int(np.ceil(0.184 * n_ids))
covariates[
    :n_cyp2p9_33+n_cyp2p9_23+n_cyp2p9_22+n_cyp2p9_13+n_cyp2p9_12, 0] += 1

typical_age = 68
covariates[:, 1] = np.random.lognormal(
    mean=np.log(typical_age), sigma=0.1, size=n_ids)

n_vkorc1_AA = int(np.ceil(0.15 * n_ids))
covariates[:n_vkorc1_AA, 2] += 1
n_vkorc1_GA = int(np.ceil(0.485 * n_ids))
covariates[:n_vkorc1_AA+n_vkorc1_GA, 2] += 1

# Sample individual-level parameters
seed = 2
data_generating_parameters = np.array([
    parameters_df[parameters_df.Parameter == p].Value.values[0]
    for p in population_model.get_parameter_names()
])
psi = population_model.sample(
    parameters=data_generating_parameters, covariates=covariates,
    n_samples=n_ids, seed=seed)

### Visualise demographic data

In [6]:
# Plot volume of distribution
fig = ff.create_distplot(
    [psi[:, 4]], ['Patients'], bin_size=.2, show_hist=True,
    show_curve=False)
fig.update_layout(xaxis_title='Volume of distribution in L')
fig.show()

# Plot clearance
n_1 = n_cyp2p9_33
n_2 = n_cyp2p9_33 + n_cyp2p9_23
n_3 = n_cyp2p9_33 + n_cyp2p9_23 + n_cyp2p9_22
n_4 = n_cyp2p9_33 + n_cyp2p9_23 + n_cyp2p9_22 + n_cyp2p9_13
n_5 = n_cyp2p9_33 + n_cyp2p9_23 + n_cyp2p9_22 + n_cyp2p9_13 + n_cyp2p9_12
fig = ff.create_distplot(
    [
        psi[:n_1, 0],
        psi[n_1:n_2, 0],
        psi[n_2:n_3, 0],
        psi[n_3:n_4, 0],
        psi[n_4:n_5, 0],
        psi[n_5:, 0]
    ],
    ['*3/*3', '*2/*3', '*2/*2', '*1/*3', '*1/*2', '*1/*1'], bin_size=.001,
    show_hist=True, show_curve=False)
fig.update_layout(xaxis_title='Elimination rate in 1/h')
fig.show()

# Plot age distribution
fig = ff.create_distplot(
    [covariates[:, 1]], ['Patients'], show_hist=True,
    show_curve=False)
fig.update_layout(xaxis_title='Age in years')
fig.show()

# Plot EC50
fig = ff.create_distplot(
    [
        psi[:n_vkorc1_AA, 1],
        psi[n_vkorc1_AA:n_vkorc1_GA, 1],
        psi[n_vkorc1_AA+n_vkorc1_GA:, 1]
    ],
    ['AA', 'GA', 'GG'], bin_size=.1, show_hist=True, show_curve=False)
fig.update_layout(xaxis_title='EC50 in mg/L')
fig.show()

### Plot steady state INR distribution

In [7]:
# Define measurement model for an individual
error_models = [
    chi.LogNormalErrorModel(),  # Warfarin concentration
    chi.LogNormalErrorModel()]  # INR
predictive_model = chi.PredictiveModel(mechanistic_model, error_models)

# Select patients
patients = np.copy(psi)
patient_cov = np.copy(covariates)

# Shuffle covariates
seed = 12
indices = np.random.choice(
    np.arange(n_ids), replace=False, size=n_ids)
patients[:, 1] = patients[indices, 1]
patient_cov[:, 0] = patient_cov[indices, 0]

# Sample INR measurements
seed = 42
times = [15 * 24]
predictive_model.set_dosing_regimen(dose=5, start=0, period=24)
inr = []
for idp, patient in enumerate(patients):
    meas = predictive_model.sample(
        parameters=patient, times=times, return_df=False, seed=seed+idp)
    inr.append(meas[1, 0, 0])

# Visualise steady state disttribution
fig = ff.create_distplot(
    [inr], ['Patients'], show_hist=True, bin_size=.1, show_curve=False)
fig.update_layout(xaxis_title='INR')
fig.show()